# Simulation of turing pattern in 1D

In [ ]:
import numpy as np

import random

from bokeh.io import  output_notebook, push_notebook, show
from bokeh.plotting import figure
from bokeh.layouts import column, row
from bokeh.palettes import Category10, Category20
from bokeh.models import PrintfTickFormatter

from ipywidgets import FloatProgress, IntProgress
from IPython.display import display

from mylib.turing_model_BZ import turing_model_BZ

import mylib.integration as integration

output_notebook(hide_banner=True)

The systeme of reaction-diffusion is the following one :

$$
\left\{
\begin{array}{l}
\partial_t y_1 =  d_1 \partial^2 _x y_1 + a - (b+1) y_1 + y_1^2y_2 \\
\partial_t y_2 =  d_2 \partial^2 _x y_2 + b y_1 - y_1^2y_2,
\end{array}
\right.
$$

with Neumann boundary conditions.

In [ ]:
def plot_sol():

    a = 2.
    b = 3.
    d1 = 1.
    d2 = 8.
    L = 3.
    xmin = 0.
    xmax = L
    # nb of points including the boundary
    nxib = 101
    nx = nxib-2
    tini = 0.0
    tend = 200.0
    neq = 2
    # "amplitude" of the initial perturbation
    eps = 0.01
    
    progress_bar = FloatProgress(min=tini, max=tend, value=tini, description='Progress:')
    display(progress_bar)
    
    tm = turing_model_BZ(a=a, b=b, d1=d1, d2=d2, xmin=xmin, xmax=xmax, nx=nx)
    fcn = tm.fcn_radau

    yini = np.zeros((nx, neq))
    x = np.linspace(xmin, xmax, nx)
    yini[:,0] = a + eps*(2*np.random.random(nx)-1)
    yini[:,1] = b/a + eps*(2*np.random.random(nx)-1)
    #yini[:,0] = a + 0.1*eps*np.cos(5*x*np.pi/L) + eps*np.cos(6*x*np.pi/L)
    #yini[:,1] = b/a + 0.1*eps*np.cos(5*x*np.pi/L) + eps*np.cos(6*x*np.pi/L)
    yini = yini.flatten()
    
    fig_sol_y1 = figure(x_range=(xmin, xmax), plot_height=300, plot_width=950, title="Solution y1")
    fig_sol_y2 = figure(x_range=(xmin, xmax), plot_height=300, plot_width=950, title="Solution y2")

    y1 = yini.reshape(nx,neq)[:,0]
    y2 = yini.reshape(nx,neq)[:,1]
    fig_sol_y1.x(x, y1, color="grey", legend = "sol at t=0")
    fig_sol_y2.x(x, y2, color="grey", legend = "sol at t=0")
    
    nt_plot = 6
    t_plot = np.linspace(tini, tend, nt_plot)
    
    for it, ti in enumerate(t_plot[:-1]):
        
        ##print(it)
        sol = integration.radau5(t_plot[it], t_plot[it+1], yini, fcn, njac=2, atol=1.e-4, rtol=1.e-4)
        y1 = sol.y.reshape(nx,neq)[:,0]
        y2 = sol.y.reshape(nx,neq)[:,1]
        fig_sol_y1.x(x, y1, legend=f"sol at t={t_plot[it+1]}", color=Category10[10][it])
        fig_sol_y2.x(x, y2, legend=f"sol at t={t_plot[it+1]}", color=Category10[10][it])        
        yini = sol.y        
        progress_bar.value = t_plot[it+1]

    fig_sol_y1.legend.click_policy="hide"
    fig_sol_y2.legend.click_policy="hide"   
    show(column(fig_sol_y1, fig_sol_y2))

plot_sol()